### First look

The idea here is to take a first look at the data and see what could infer from the same. Based on these inferences, we might create transformations and build models on the same.


In [2]:
import pandas as pd
import numpy as np


#### Start with item_categories

In [3]:
item_categories = pd.read_csv("../input/item_categories.csv")

In [4]:
len(item_categories)

84

In [5]:
item_categories.isna().any()

item_category_name    False
item_category_id      False
dtype: bool

In [6]:
item_categories.head(10)

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4
5,Аксессуары - PSVita,5
6,Аксессуары - XBOX 360,6
7,Аксессуары - XBOX ONE,7
8,Билеты (Цифра),8
9,Доставка товара,9


#### Now with shops data

In [7]:
shops = pd.read_csv("../input/shops.csv")

In [8]:
len(shops)

60

In [9]:
shops.isna().any()

shop_name    False
shop_id      False
dtype: bool

In [10]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


#### Items data

In [11]:
items = pd.read_csv("../input/items.csv")

In [12]:
len(items)

22170

In [13]:
items.columns

Index(['item_name', 'item_id', 'item_category_id'], dtype='object')

In [14]:
items.isna().any()

item_name           False
item_id             False
item_category_id    False
dtype: bool

In [15]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


#### And finally, the train data

In [16]:
sales_train = pd.read_csv("../input/sales_train.csv")

In [17]:
len(sales_train)

2935849

In [18]:
sales_train.columns

Index(['date', 'date_block_num', 'shop_id', 'item_id', 'item_price',
       'item_cnt_day'],
      dtype='object')

In [19]:
sales_train.isna().any()

date              False
date_block_num    False
shop_id           False
item_id           False
item_price        False
item_cnt_day      False
dtype: bool

In [20]:
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


#### Let us take a look at all entries for shop_id 59

In [21]:
sales_train[sales_train['shop_id'] == 59].head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.0,1.0
40084,10.01.2013,0,59,22151,399.0,1.0
77502,04.01.2013,0,59,5603,699.0,1.0
77503,19.01.2013,0,59,5587,199.0,2.0
77504,31.01.2013,0,59,5613,5571.0,1.0


##### How many shops are present in the training data ? Are there shops present in test data that are not present in training data ?

In [22]:
shops_train_data = set(sales_train['shop_id'].values)

In [23]:
test = pd.read_csv('../input/test.csv')

In [24]:
shops_test_data = set(test['shop_id'].values)

In [25]:
shops_train_data - shops_test_data

{0, 1, 8, 9, 11, 13, 17, 20, 23, 27, 29, 30, 32, 33, 40, 43, 51, 54}

In [26]:
shops_test_data - shops_train_data

set()

### Separate into train, test and validation sets.

Now, that we have taken a look at the data, let us split into train, validation and test sets so that we could do serious model building using the same.


In [27]:
# Pandas groubby count per month to see entries per month.
monthly_data = sales_train[['date', 'date_block_num' ]].groupby(['date_block_num']).count()

In [28]:
monthly_data.head()

,date
date_block_num,
0,115690
1,108613
2,121347
3,94109
4,91759


In [29]:
len(sales_train)

2935849

In [30]:
len(sales_train[sales_train['date_block_num'] == 32])/len(sales_train)

0.017231131437618216

In [31]:
# Decide on training/validation/test sets based on the output from previous step.
# To make sure that training using time series data makes sense, we take the data for month 32 as the validation set
# and the data for month 33 as the test set.

train_data = sales_train[(sales_train['date_block_num'] != 32) &(sales_train['date_block_num'] != 33)]
validation_data = sales_train[sales_train['date_block_num'] == 32]
test_data = sales_train[sales_train['date_block_num'] == 33]

#### Transform daily data to monthly data and see how the simple model works



##### Transforming to monthly data

In [32]:
monthly_train_data = train_data[['date_block_num', 'shop_id', 'item_id', 'item_cnt_day' ]].groupby(
    ['date_block_num', 'shop_id', 'item_id']).sum()

In [33]:
monthly_train_data.head()

item_cnt_day
date_block_num shop_id item_id              
0              0       32                6.0
                       33                3.0
                       35                1.0
                       43                1.0
                       51                2.0

In [34]:
monthly_train_data.reset_index(['date_block_num', 'shop_id', 'item_id'], inplace=True)

In [35]:
monthly_train_data.rename(columns={'item_cnt_day':'item_cnt_month'}, inplace=True)

In [36]:
monthly_validation_data = validation_data[['date_block_num', 'shop_id', 'item_id', 'item_cnt_day' ]].groupby(
    ['date_block_num', 'shop_id', 'item_id']).sum()

In [37]:
monthly_validation_data.reset_index(['date_block_num', 'shop_id', 'item_id'], inplace=True)

In [38]:
monthly_validation_data.rename(columns={'item_cnt_day':'item_cnt_month'}, inplace=True)

In [39]:
monthly_test_data = test_data[['date_block_num', 'shop_id', 'item_id', 'item_cnt_day' ]].groupby(
    ['date_block_num', 'shop_id', 'item_id']).sum()

monthly_test_data.reset_index(['date_block_num', 'shop_id', 'item_id'], inplace=True)
monthly_test_data.rename(columns={'item_cnt_day':'item_cnt_month'}, inplace=True)

In [40]:
monthly_train_data[(monthly_train_data['shop_id'] == 0) & (monthly_train_data['item_id'] == 32)]

,date_block_num,shop_id,item_id,item_cnt_month
0,0,0,32,6.0
63226,1,0,32,10.0


In [41]:
train_data[(train_data['shop_id'] == 0) & (train_data['item_id'] == 32)]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
40085,03.01.2013,0,0,32,221.0,2.0
40086,21.01.2013,0,0,32,221.0,2.0
40087,25.01.2013,0,0,32,221.0,1.0
40088,31.01.2013,0,0,32,221.0,1.0
173447,25.02.2013,1,0,32,221.0,1.0
173448,22.02.2013,1,0,32,221.0,2.0
173449,16.02.2013,1,0,32,221.0,2.0
173450,15.02.2013,1,0,32,221.0,1.0
173452,12.02.2013,1,0,32,221.0,2.0
173453,05.02.2013,1,0,32,221.0,1.0


In [42]:
monthly_train_data[(monthly_train_data['shop_id'] == 2) & (monthly_train_data['item_id'] == 792)]

,date_block_num,shop_id,item_id,item_cnt_month
975031,18,2,792,1.0
1020786,19,2,792,3.0
1067228,20,2,792,1.0
1107652,21,2,792,1.0
1255271,24,2,792,1.0
1302053,25,2,792,2.0
1416775,28,2,792,1.0
1448994,29,2,792,1.0
1514432,31,2,792,2.0


In [43]:
train_data[(train_data['shop_id'] == 2) & (train_data['item_id'] == 792)]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
1845134,05.07.2014,18,2,792,979.0,1.0
1924816,23.08.2014,19,2,792,979.0,1.0
1924817,07.08.2014,19,2,792,979.0,1.0
1924818,05.08.2014,19,2,792,979.0,1.0
1956091,04.09.2014,20,2,792,979.0,1.0
2052761,05.10.2014,21,2,792,979.0,1.0
2410959,03.01.2015,24,2,792,979.0,1.0
2459334,15.02.2015,25,2,792,979.0,1.0
2459344,21.02.2015,25,2,792,979.0,1.0
2621170,06.05.2015,28,2,792,979.0,1.0


##### Getting an average of monthly data

In [44]:
monthly_train_data_avg = monthly_train_data[['shop_id', 'item_id', 'item_cnt_month']].groupby(['shop_id', 'item_id']).mean()

In [45]:
monthly_train_data_avg.reset_index(['shop_id', 'item_id'], inplace=True)

In [46]:
monthly_train_data_avg.rename(columns={'item_cnt_month':'item_cnt_month_avg'}, inplace=True)

In [47]:
monthly_train_data_avg.head()

,shop_id,item_id,item_cnt_month_avg
0,0,30,31.0
1,0,31,11.0
2,0,32,8.0
3,0,33,3.0
4,0,35,7.5


In [48]:
monthly_train_data_avg[(monthly_train_data_avg['shop_id'] == 0) & (monthly_train_data_avg['item_id'] == 32)]

,shop_id,item_id,item_cnt_month_avg
2,0,32,8.0


In [49]:
monthly_train_data_avg[(monthly_train_data_avg['shop_id'] == 2) & (monthly_train_data_avg['item_id'] == 792)]

,shop_id,item_id,item_cnt_month_avg
6170,2,792,1.444444


In [50]:
monthly_train_data_avg['shop_id_and_item_id'] = list(zip(monthly_train_data_avg['shop_id'], 
                                                         monthly_train_data_avg['item_id']))

In [51]:
shop_id_item_id_item_cnt_monthly_avg = dict(zip(monthly_train_data_avg.shop_id_and_item_id, 
                                                monthly_train_data_avg.item_cnt_month_avg))

In [52]:
shop_id_item_id_item_cnt_monthly_avg[(2, 792)]

1.4444444444444444

##### Getting predictions for entries in the validation data set


In [53]:
monthly_validation_data['item_cnt_month_prediction'] = monthly_validation_data.apply(
    lambda x : shop_id_item_id_item_cnt_monthly_avg.get((x.shop_id, x.item_id), -1), axis=1) 

In [54]:
monthly_validation_data.head(15)

,date_block_num,shop_id,item_id,item_cnt_month,item_cnt_month_prediction
0,32,2,33,1.0,1.111111
1,32,2,486,1.0,1.714286
2,32,2,792,1.0,1.444444
3,32,2,975,1.0,1.000000
4,32,2,1090,1.0,1.000000
5,32,2,1108,1.0,2.000000
6,32,2,1250,1.0,-1.000000
7,32,2,1288,1.0,1.000000
8,32,2,1384,1.0,2.000000
9,32,2,1467,2.0,1.416667


##### Handling the entries in validation set when no corresponding entries are present in training set.

In [55]:
len(monthly_validation_data[monthly_validation_data['item_cnt_month_prediction'] == -1])

5164

In [56]:
len(monthly_validation_data)

29678

In [57]:
# Since there are sizeable number of entries, let us remove them before computing the prediction score.
monthly_validation_data_processed = monthly_validation_data[(monthly_validation_data['item_cnt_month_prediction'] != -1)].copy()

In [58]:
len(monthly_validation_data_processed)

24514

In [59]:
from sklearn.metrics import mean_squared_error
rmse_validation_data = np.sqrt(mean_squared_error(monthly_validation_data_processed.item_cnt_month, 
                                                  monthly_validation_data_processed.item_cnt_month_prediction))

In [60]:
rmse_validation_data

19.314513082177427

##### The simple model looks terrible (Well that is the reason as to why we have this problem in the first place). Let us dump data into separate files and continue our analysis in a separate notebook.

##### Oh no, we actually missed the clipping part here (the values are clipped into [0, 20] range). Let us do that and see how our performance improves !

In [61]:
monthly_validation_data_processed['item_cnt_month'] = \
    monthly_validation_data_processed['item_cnt_month'].apply(lambda x : np.clip(x, 0, 20))

In [62]:
monthly_validation_data_processed['item_cnt_month_prediction'] = \
    monthly_validation_data_processed['item_cnt_month_prediction'].apply(lambda x : np.clip(x, 0, 20))

In [63]:
rmse_validation_data = np.sqrt(mean_squared_error(monthly_validation_data_processed.item_cnt_month, 
                                                  monthly_validation_data_processed.item_cnt_month_prediction))

In [64]:
rmse_validation_data

2.851393797271763

In [73]:
# Put the clippings to a routine for easy access.
def clip_predictions_and_actual_vals(df, 
                                     actual_value_col='item_cnt_month', 
                                     prediction_col='item_cnt_month_prediction'):
    df[actual_value_col] = \
        df[actual_value_col].apply(lambda x : np.clip(x, 0, 20))
    df[prediction_col] = \
        df[prediction_col].apply(lambda x : np.clip(x, 0, 20))    
    


##### To do

1. Check out performance of models which always output 1 or 0.5 (after clipping the data)
2. Check out performance when we average if we have data and use the default value if we do not (again,after clipping data).



##### When we set all predictions to 1

In [74]:
monthly_validation_data['item_cnt_month_prediction'] = 1
clip_predictions_and_actual_vals(monthly_validation_data)
rmse_validation_data = np.sqrt(mean_squared_error(monthly_validation_data.item_cnt_month, 
                                                  monthly_validation_data.item_cnt_month_prediction))    
rmse_validation_data

2.727832006809773

##### When we set all predictions to 0.5

In [75]:
monthly_validation_data['item_cnt_month_prediction'] = 0.5
clip_predictions_and_actual_vals(monthly_validation_data)
rmse_validation_data = np.sqrt(mean_squared_error(monthly_validation_data.item_cnt_month, 
                                                  monthly_validation_data.item_cnt_month_prediction))    
rmse_validation_data

2.938422210862784

##### Take the average of historical data if available, and if not default to 1.

In [76]:
monthly_validation_data['item_cnt_month_prediction'] = monthly_validation_data.apply(
    lambda x : shop_id_item_id_item_cnt_monthly_avg.get((x.shop_id, x.item_id), -1), axis=1) 

monthly_validation_data.loc[(monthly_validation_data['item_cnt_month_prediction'] == -1), 
                            'item_cnt_month_prediction'] = 1
clip_predictions_and_actual_vals(monthly_validation_data)
rmse_validation_data = np.sqrt(mean_squared_error(monthly_validation_data.item_cnt_month, 
                                                  monthly_validation_data.item_cnt_month_prediction)) 
rmse_validation_data


3.0715229511309157

##### Default to 0.5 instead of 1

In [77]:
monthly_validation_data['item_cnt_month_prediction'] = monthly_validation_data.apply(
    lambda x : shop_id_item_id_item_cnt_monthly_avg.get((x.shop_id, x.item_id), -1), axis=1) 

monthly_validation_data.loc[(monthly_validation_data['item_cnt_month_prediction'] == -1), 
                            'item_cnt_month_prediction'] = 0.5
clip_predictions_and_actual_vals(monthly_validation_data)
rmse_validation_data = np.sqrt(mean_squared_error(monthly_validation_data.item_cnt_month, 
                                                  monthly_validation_data.item_cnt_month_prediction)) 
rmse_validation_data


3.1184079613448312

#### Comments

Interestingly, it looks like defaulting the predictions to 1, does a seemingly better job than setting them to the historical average. Let us take a closer look in the next notebook.

In [66]:
train_data.to_csv('../processed_input/train_data.csv', index=False)
validation_data.to_csv('../processed_input/validation_data.csv', index=False)
test_data.to_csv('../processed_input/test_data.csv', index=False)

monthly_train_data.to_csv('../processed_input/monthly_train_data.csv', index=False)
monthly_train_data_avg.to_csv('../processed_input/monthly_train_data_avg.csv', index=False)

monthly_validation_data.to_csv('../processed_input/monthly_validation_data.csv', index=False)
monthly_test_data.to_csv('../processed_input/monthly_test_data.csv', index=False)